In [1]:
!pip install -q sentence-transformers faiss-cpu pandas numpy


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
!pip install tf-keras


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import os
os.environ["USE_TF_KERAS"] = "1"

import pandas as pd
import numpy as np
import pickle
from sentence_transformers import SentenceTransformer
import faiss


df = pd.read_csv("product_info_skincare.csv")
print("✅ Dataset Loaded:", df.shape)
print("Columns:", df.columns.tolist())
df.head()

df['description'] = (
    df['product_name'].fillna('') + " | " +
    df['ingredients'].fillna('') + " | " +
    df['primary_category'].fillna('') + " | " +
    df['highlights'].fillna('')
)

print("✅ Created description column for embeddings")


model = SentenceTransformer('all-MiniLM-L6-v2')
print("✅ Model Loaded")

product_embeddings = model.encode(
    df['description'].astype(str).tolist(),
    show_progress_bar=True,
    convert_to_numpy=True
)
print("✅ Embeddings Shape:", product_embeddings.shape)

dimension = product_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(product_embeddings)
print("✅ FAISS index is ready with", index.ntotal, "items")

with open("product_embeddings.pkl", "wb") as f:
    pickle.dump(product_embeddings, f)

faiss.write_index(index, "product_faiss.index")
df.to_csv("product_info_processed.csv", index=False)

print("✅ Files saved for Streamlit:")
print("   - product_embeddings.pkl")
print("   - product_faiss.index")
print("   - product_info_processed.csv")

def recommend_products(query, top_n=5):
    query_embedding = model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, top_n)
    results = df.iloc[indices[0]].copy()
    results['distance'] = distances[0]

    display_cols = [c for c in ['product_name', 'price_usd', 'rating', 'distance'] if c in results.columns]
    return results[display_cols]

test_query = input("search: ")
print("🔹 Top 5 Recommendations for:", test_query)
recommend_products(test_query, top_n=5)

ValueError: Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [ ]:
import nbformat

path = "D:\skin care\main_model.ipynb"
nb = nbformat.read(path, as_version=nbformat.NO_CONVERT)

if 'widgets' in nb.get('metadata', {}):
    del nb['metadata']['widgets']

nbformat.write(nb, path)
print("✅ Cleaned notebook metadata. You can now upload to GitHub.")